In [ ]:
# Develop SMARTS

Just a playground notebook to check some ideas for substructure SMARTS or reactionSMARTS for pattern matching / enumeration in SynFerm libraries

In [ ]:
import pathlib
import sys

import pandas as pd
from rdkit import Chem
from rdkit.Chem import ChiralType
from rdkit.Chem import Draw
from rdkit.Chem.rdChemReactions import ReactionFromSmarts

sys.path.append(str(pathlib.Path().resolve().parents[1]))
from src.util.db_utils import SynFermDatabaseConnection

In [ ]:
# get building blocks
con = SynFermDatabaseConnection()
res = con.con.execute("SELECT * FROM building_blocks").fetchall()
header = [i[1] for i in con.con.execute("PRAGMA table_info(building_blocks)").fetchall()]
df = pd.DataFrame(res, columns=header)
initiators = df.loc[df["category"] == "I"]
monomers = df.loc[df["category"] == "M"]
terminators = df.loc[df["category"] == "T"]

In [ ]:
mol = Chem.MolFromSmiles("COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cc(Cl)ccn3)[C@H](c3nc4ccccc4s3)C2)cc1OC")
mol

In [ ]:
pat = Chem.MolFromSmarts("[#6:1]-[#6](=O)-[NR0]-[#6:2]-[#6:3]-c1nc2[c:4][c:5][c:6][c:7]c2s1")
pat

In [ ]:
mol.GetSubstructMatches(pat)

In [ ]:
mol

In [ ]:
Chem.MolFromSmiles("COc1ccc(CCOC(=O)N2C[C@@H]3NOC4(OC5(CCCCC5)OC4=O)[C@@H]3C2)cc1OC")

In [ ]:
Chem.MolFromSmiles("COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)OC4=O)[C@@H]3C2)cc1OC.Cl")

In [ ]:
mol = Chem.MolFromSmiles("COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)OC4=O)[C@@H]3C2)cc1OC")
mol

In [ ]:
pat = Chem.MolFromSmarts("[$([CR2](O1)(ONC2)(C2)C(=O)OC1)]")  # hits a spiro carbon atom between an isoxazolidine and a 5-membered lactone-ether ring
pat

In [ ]:
mol.GetSubstructMatches(pat)

In [ ]:
a = mol.GetAtomWithIdx(16)

In [ ]:
a.SetChiralTag(ChiralType.CHI_UNSPECIFIED)

In [ ]:
Chem.MolToSmiles(mol) == "COc1ccc(CCOC(=O)N2C[C@@H]3NOC4(OC5(CCCCC5)OC4=O)[C@@H]3C2)cc1OC"

In [ ]:
def remove_monomer_pg_chirality(mol):
    """
    Given a monomer building block, remove only the chiral information 
    for the spiro carbon of the protecting group, leaving the rest unchanged.
    """
    if isinstance(mol, str):
        mol = Chem.MolFromSmiles(mol)
    else:
        mol = Chem.Mol(mol)
    match = mol.GetSubstructMatches(pat)
    assert len(match) == 1
    atom = mol.GetAtomWithIdx(match[0][0])
    atom.SetChiralTag(ChiralType.CHI_UNSPECIFIED)
    return mol

In [ ]:
Draw.MolsToGridImage([remove_monomer_pg_chirality(smi) for smi in monomers["SMILES"]])

In [ ]:
con.get_vl_member(19)

In [ ]:
_backwards_rxn_abt = "[#6:1]-C(=O)-N-[C:2]-[C:3]-c1nc2[c:4][c:5][c:6][c:7]c2s1>>F-[B-](-F)(-F)-C(-[#6:1])=O.O=C1-O-C2(-C-C-C-C-C-2)-O-C-1-1-[C:3]-[C:2]-N-O-1.N-c1[c:4][c:5][c:6][c:7]c1-S"
rxn = ReactionFromSmarts(_backwards_rxn_abt)
rxn

In [ ]:
res = rxn.RunReactants((con.get_vl_member(18),))[0]
[Chem.SanitizeMol(m) for m in res]

In [ ]:
Draw.MolsToGridImage(res)

In [ ]:
Chem.MolFromSmiles("NNC(=S)/C=C/c1ccccc1")

In [ ]:
Chem.MolFromSmiles("NNC(=S)C=Cc1ccccc1")

In [ ]:
Chem.MolToSmiles(con.get_vl_member(25))

In [ ]:
pat

In [ ]:
mol = con.get_vl_member(288)
mol.GetSubstructMatches(pat)
mol

In [ ]:
con.get_vl_member(54051)

In [ ]:
rxn = ReactionFromSmarts(
    "[$(B(-F)(-F)-F)]-[C:2](-[#6:1])=[O:3].O=C1-O-[$(C2CCCCC2)]-O-[C:7]-1-1-[C:6]-[C:5]-[N:4]-O-1.[N:8]-[c:9]1:[c:10]:[c:11]:[c:12]:[c:13]:[c:14]:1-[S:15]>>[#6:1]-[C:2](=[O:3])-[N:4]-[C:5]-[C:6]-[c:7]1:[n:8]:[c:9]2:[c:10]:[c:11]:[c:12]:[c:13]:[c:14]:2:[s:15]:1."
)
rxn

In [ ]:
rxn = ReactionFromSmarts(
"[$(B(-F)(-F)-F)]-[C:2](-[#6:1])=[O:3].O=C1-O-[$(C2CCCCC2)]-O-[C:7]-1-1-[C:6]-[C:5]-[N:4]-O-1.[#6:8]-[C:9](=[S:10])-[N:11]-[N:12]>>[#6:8]-[c:9]1:[n:11]:[n:12]:[c:7](-[C:6]-[C:5]-[N:4]-[C:2](-[#6:1])=[O:3]):[s:10]:1."
)
rxn

In [ ]:
i = Chem.MolFromSmiles(con.get_smiles("I2"))
i

In [ ]:
m = Chem.MolFromSmiles(con.get_smiles("M2"))
m

In [ ]:
t = Chem.MolFromSmiles(con.get_smiles("T20"))
t

In [ ]:
rxn

In [ ]:
prods = rxn.RunReactants((i,m,t))

In [ ]:
prods[0][0]

In [ ]:
prods

In [ ]:
pat = rxn.GetReactantTemplate(1)

m.GetSubstructMatches(pat)
m

In [ ]:
smi = "F[B-](F)(F)[C:2]([c:1]1[cH:13][cH:15][cH:18][cH:17][c:14]1[CH3:16])=[O:3].O=C1OC2(CCCCC2)O[C:7]12O[NH:4][C:5]1([CH2:6]2)[CH2:19][S:21][CH2:20]1.[c:8]1([C:9](=[S:10])[NH:11][NH2:12])[cH:22][cH:24][c:26]2[c:25]([cH:23]1)[O:27][CH2:29][O:28]2>>[c:1]1([C:2](=[O:3])[NH:4][C:5]2([CH2:6][c:7]3[s:10][c:9](-[c:8]4[cH:22][cH:24][c:26]5[c:25]([cH:23]4)[O:27][CH2:29][O:28]5)[n:11][n:12]3)[CH2:19][S:21][CH2:20]2)[cH:13][cH:15][cH:18][cH:17][c:14]1[CH3:16]"

rxn = ReactionFromSmarts(smi, useSmiles=True)
rxn

In [ ]:
df = pd.read_csv("../../data/curated_data/synferm_dataset_2023-09-05_40018records.csv")[["product_A_smiles", "reaction_smiles_atom_mapped", "experiment_id"]]

In [ ]:
Chem.MolFromSmiles(df.loc[df.experiment_id == "56113", "product_A_smiles"].item())

In [ ]:
Chem.MolToSmiles(con.get_vl_member(35080))

In [ ]:
# reactants to product H
_rxn_abt_H = "[$(B(-F)(-F)-F)]-[$(C-[#6])X3:1]=[O:2].O=C1-O-[$(C2CCCCC2)]-O-[C:6]-1-1-[C:5](-[H])-[C:4]-[NH1:3]-O-1.[NH2:7]-[c:8]1:[c:9]:[c:10]:[c:11]:[c:12]:[c:13]:1-[SH1:14]>>[C:4]=[C:5]-[c:6]:1:[n:7]:[c:8]2:[c:9]:[c:10]:[c:11]:[c:12]:[c:13]:2:[s:14]:1"
_rxn_th_H = "[$(B(-F)(-F)-F)]-[$(C-[#6])X3:1]=[O:2].O=C1-O-[$(C2CCCCC2)]-O-[C:6]-1-1-[C:5](-[H])-[C:4]-[NH1:3]-O-1.[C:7](=[S:8])-[NH1:9]-[NH2:10]>>[s:8]:1:[c:7]:[n:9]:[n:10]:[c:6]:1-[C:5]=[C:4]"


In [ ]:
rxn = ReactionFromSmarts(_rxn_abt_H)
rxn

In [ ]:
rxn = ReactionFromSmarts(_rxn_th_H)
rxn

In [ ]:
initiators.loc[initiators.long == "2-Pyr002"]

In [ ]:
monomers.loc[monomers.long == "Fused002"]

In [ ]:
terminators.loc[terminators.long == "TerABT001"]

In [ ]:
Chem.AddHs(desalt_building_block(Chem.MolFromSmiles(monomers.at[134, "SMILES"])))

In [ ]:
terminators.at[78, "SMILES"]

In [ ]:
from rdkit.Chem.rdChemReactions import SanitizeRxn

In [ ]:
rxn.Initialize()
#SanitizeRxn(rxn)

In [ ]:
rxn

In [ ]:
prods = rxn.RunReactants(
    [desalt_building_block(Chem.MolFromSmiles(initiators.at[3, "SMILES"])), 
     Chem.AddHs(desalt_building_block(Chem.MolFromSmiles(monomers.at[78, "SMILES"]))), 
     desalt_building_block(Chem.MolFromSmiles(terminators.at[152, "SMILES"]))
    ]
)

In [ ]:
prods

In [ ]:
p = prods[0][0]
p

In [ ]:
Chem.rdmolops.RemoveAllHs(p)

In [ ]:
p = Chem.rdmolops.RemoveHs(p)

In [ ]:
Chem.SanitizeMol(p)

In [ ]:
p

In [ ]:
Chem.MolFromSmiles('[H]C1=[C@]([H])(c2nc3ccccc3s2)C([H])([H])N(C(=O)OC([H])([H])C([H])([H])c2c([H])c([H])c(OC([H])([H])[H])c(OC([H])([H])[H])c2[H])C1([H])[H]', sanitize=False)

In [ ]:
Chem.MolToSmiles(p)

In [ ]:
p.UpdatePropertyCache()

In [ ]:
p.GetAtomWithIdx(5).GetTotalValence()

In [ ]:
p.GetAtomWithIdx(5).GetNumExplicitHs()

In [ ]:
p = Chem.rdmolops.AddHs(p)

In [ ]:
p

In [ ]:
p = Chem.rdmolops.RemoveHs(p, sanitize=False, updateExplicitCount=True)

In [ ]:
Chem.rdmolops.Cleanup(p)

In [ ]:
Chem.SanitizeMol(p)

In [ ]:
a.GetNeighbors()

In [ ]:
Chem.MolToSmiles(Chem.MolFromSmiles("C(=Cc1cccnc1)c1nnc(-c2cccc3[nH]ncc23)s1"))

In [ ]:
Chem.MolToSmiles(Chem.MolFromSmiles("C(=Cc1nnc(-c2cccc3[nH]ncc23)s1)c1cccnc1"))

In [ ]:
from src.library_design.reaction_generator import SFReactionGenerator

In [ ]:
rxn_generator = SFReactionGenerator()

In [ ]:
[Chem.MolToSmiles(smi) for smi in rxn_generator.generate_reactants(Chem.MolToSmiles(con.get_vl_member(35080)))]

In [ ]:
rxn_generator.backwards_reactions["th"]

In [ ]:
_backwards_rxn_abt = "[$(C-[#6]):1](=O)-[NR0]-[C:2]-[C:3]-c1nc2[c:4][c:5][c:6][c:7]c2s1>>F-[B-](-F)(-F)-[C:1]=O.O=C1-O-C2(-C-C-C-C-C-2)-O-C-1-1-[C:3]-[C:2]-N-O1.N-c1:[c:4]:[c:5]:[c:6]:[c:7]:c:1-S"


In [ ]:
_backwards_rxn_th = "[c:4]1nnc(-[C:3]-[C:2]-[NR0]-[$(C-[#6]):1]=O)s1>>F-[B-](-F)(-F)-[C:1]=O.O=C1-O-C2(-C-C-C-C-C-2)-O-C-1-1-[C:3]-[C:2]-N-O1.[C:4](=S)-N-N"


In [ ]:
rxn = ReactionFromSmarts(_backwards_rxn_abt)
rxn

In [ ]:
prods = rxn.RunReactants((con.get_vl_member(35080),))

In [ ]:
prods[0][0]

In [ ]:
_rxn_abt = "[$(B(-F)(-F)-F)]-[$(C-[#6])X3:1]=[O:2].O=C1-O-[$(C2CCCCC2)]-O-[C:6]-1-1-[C:5]-[C:4]-[NH1:3]-O-1.[NH2:7]-[c:8]1:[c:9]:[c:10]:[c:11]:[c:12]:[c:13]:1-[SH1:14]>>[C:1](=[O:2])-[N:3]-[C:4]-[C:5]-[c:6]1:[n:7]:[c:8]2:[c:9]:[c:10]:[c:11]:[c:12]:[c:13]:2:[s:14]:1."
rxn = ReactionFromSmarts(_rxn_abt)

In [ ]:
_rxn_th = "[$(B(-F)(-F)-F)]-[$(C-[#6])X3:1]=[O:2].O=C1-O-[$(C2CCCCC2)]-O-[C:6]-1-1-[C:5]-[C:4]-[NH1:3]-O-1.[C:7](=[S:8])-[NH1:9]-[NH2:10]>>[c:7]1:[n:9]:[n:10]:[c:6](-[C:5]-[C:4]-[N:3]-[C:1]=[O:2]):[s:8]:1."
rxn = ReactionFromSmarts(_rxn_th)

In [ ]:
rxn

In [ ]:
from src.util.rdkit_util import desalt_building_block

In [ ]:
for atom in rxn.GetReactantTemplate(0).GetAtoms():
    print(atom.GetSymbol())

In [ ]:
rxn.Initialize()

In [ ]:
Chem.rdChemReactions.SanitizeRxn(rxn)

In [ ]:
# Prepare atom indices for drawing
for i, atom in enumerate(mol.GetAtoms()):
    atom.SetProp("molAtomMapNumber", str(atom.GetIdx()))
mol

In [ ]:
prods = rxn.RunReactants(
    [desalt_building_block(Chem.MolFromSmiles(initiators.at[0, "SMILES"])), 
     desalt_building_block(Chem.MolFromSmiles(monomers.at[78, "SMILES"])), 
     desalt_building_block(Chem.MolFromSmiles(terminators.at[172, "SMILES"]))
    ]
)

In [ ]:
mol = prods[0][0]
Chem.SanitizeMol(mol)

In [ ]:
mol

In [ ]:
Chem.MolFromSmiles(initiators.at[0, "SMILES"])

In [ ]:
terminators

In [ ]:
ReactionFromSmarts(
    "F[B-](F)(F)[C:2]([c:1]1[cH:13][cH:15][cH:17][c:16]([O:18][CH3:19])[n:14]1)=[O:3].O=C1OC2(CCCCC2)O[C:7]12O[NH:4][CH2:5][C:6]21[CH2:20][CH2:22][CH2:21]1.[c:8]1([C:9](=[S:10])[NH:11][NH2:12])[cH:23][n:25][nH:26][cH:24]1>>[c:1]1([C:2](=[O:3])[NH:4][CH2:5][C:6]2([c:7]3[s:10][c:9](-[c:8]4[cH:23][n:25][nH:26][cH:24]4)[n:11][n:12]3)[CH2:20][CH2:22][CH2:21]2)[cH:13][cH:15][cH:17][c:16]([O:18][CH3:19])[n:14]1",
    useSmiles=True
)

In [ ]:
ReactionFromSmarts(
    "F[B-](F)(F)[C:1](=[O:2])[c:11]1[cH:12][cH:14][cH:16][c:15]([O:17][CH3:18])[n:13]1.O=C1OC2(CCCCC2)O[C:6]12O[NH:3][CH2:4][C:5]21[CH2:19][CH2:21][CH2:20]1.[C:7](=[S:8])([NH:9][NH2:10])[c:22]1[cH:23][n:25][nH:26][cH:24]1>>[C:1](=[O:2])([NH:3][CH2:4][C:5]1([c:6]2[s:8][c:7](-[c:22]3[cH:23][n:25][nH:26][cH:24]3)[n:9][n:10]2)[CH2:19][CH2:21][CH2:20]1)[c:11]1[cH:12][cH:14][cH:16][c:15]([O:17][CH3:18])[n:13]1",
    useSmiles=True
)